In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK080879,Karma Resorts,Other,Past Guests,Karma Kandara,PF332062,Mr.,Henry Ted,Sinclair,Male,...,2025-10-12 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
1,KK080601,Karma Resorts,Other,Past Guests,Karma Kandara,PF331283,Mr.,Jason Ray,Ie,Male,...,2025-10-17 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KB053883,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332279,Mrs,Jennie,Toyne,Female,...,2025-10-17 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
3,KK074344,Karma Resorts,Other,Past Guests,Karma Kandara,PF332393,Mr.,Leroy Joseph,Briggs,Male,...,2025-10-13 00:00:00,2,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KK070080,Karma Resorts,Other,Past Guests,Karma Kandara,PF304383,Mr,Martin,Szczerbiak,Male,...,2025-10-12 00:00:00,6,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,KK073443,Karma Resorts,Other,Past Guests,Karma Kandara,PF311650,Mr,David,Cowham,Male,...,2025-10-18 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
77,KSM016430,Karma Resorts,Other,Past Guests,Karma St Martins,PF322878,Miss,Terrie Tucker,Holmes,Unknown,...,2025-10-13 00:00:00,2,COMP,STAFF,COMP,2,0,0,Phone Only,NO
78,KSM017268,Karma Resorts,Other,Past Guests,Karma St Martins,PF333222,Mrs,Debi,Peskett,Unknown,...,2025-10-15 00:00:00,1,DIR,WALKIN,RETAIL,1,0,0,Phone Only,NO
79,KB053882,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332646,Mrs,Jennifer,Ribley,Female,...,2025-10-12 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
9,KB052459,Karma Resorts,Other,Past Guests,Karma Bavaria,PF324654,Herr,Stefan,Moser,Male,...,2025-10-12 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
19,KB053964,Karma Resorts,Other,Past Guests,Karma Bavaria,PF313031,Frau,Elke,Wachenfeld-Teschner,Female,...,2025-10-12 10:30:00,0,DIR,EMAIL_MRKTNG,DIGITAL,1,0,0,Email and Phone,NO
21,KB053959,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332973,Herr,Joachim,Lochmann,Male,...,2025-10-13 00:00:00,1,COMP,GSTAFF,COMP,1,0,0,Email Only,NO
34,KB053469,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329930,Frau,Vanessa,Bochler,Female,...,2025-10-14 19:30:00,0,CB,TEL,RETAIL,1,0,0,Email and Phone,NO
36,KB054060,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290977,Herr,Werner,Wiemken,Male,...,2025-10-17 15:00:00,0,CORP,LGR,GROUP,1,0,0,Email and Phone,NO
44,KB053547,Karma Resorts,Other,Past Guests,Karma Bavaria,PF330329,empty,Nadja,Schittko,Unknown,...,2025-10-12 00:00:00,4,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
48,KK080098,Karma Resorts,Other,Past Guests,Karma Kandara,PF320024,Mr,.,Made,Male,...,2025-10-13 00:00:00,1,HOUSE_USE,GSTAFF,COMP,2,0,0,Email and Phone,NO
72,KSM015422,Karma Resorts,Other,Past Guests,Karma St Martins,PF289035,Miss,Tia,Brown,Unknown,...,2025-10-13 00:00:00,2,COMP,WALKIN,RETAIL,2,0,0,Email and Phone,NO
73,KSM014154,Karma Resorts,Other,Past Guests,Karma St Martins,PF289035,Miss,Tia,Brown,Unknown,...,2025-10-13 00:00:00,2,COMP,TEL,RETAIL,2,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK080879,Karma Resorts,Other,Past Guests,Karma Kandara,PF332062,Mr.,Henry Ted,Sinclair,Male,...,2025-10-12 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
1,KK080601,Karma Resorts,Other,Past Guests,Karma Kandara,PF331283,Mr.,Jason Ray,Ie,Male,...,2025-10-17 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KB053883,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332279,Mrs,Jennie,Toyne,Female,...,2025-10-17 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
3,KK074344,Karma Resorts,Other,Past Guests,Karma Kandara,PF332393,Mr.,Leroy Joseph,Briggs,Male,...,2025-10-13 00:00:00,2,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KK070080,Karma Resorts,Other,Past Guests,Karma Kandara,PF304383,Mr,Martin,Szczerbiak,Male,...,2025-10-12 00:00:00,6,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,KK073443,Karma Resorts,Other,Past Guests,Karma Kandara,PF311650,Mr,David,Cowham,Male,...,2025-10-18 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
68,KSM016430,Karma Resorts,Other,Past Guests,Karma St Martins,PF322878,Miss,Terrie Tucker,Holmes,Unknown,...,2025-10-13 00:00:00,2,COMP,STAFF,COMP,2,0,0,Phone Only,NO
69,KSM017268,Karma Resorts,Other,Past Guests,Karma St Martins,PF333222,Mrs,Debi,Peskett,Unknown,...,2025-10-15 00:00:00,1,DIR,WALKIN,RETAIL,1,0,0,Phone Only,NO
70,KB053882,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332646,Mrs,Jennifer,Ribley,Female,...,2025-10-12 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
19,KB053675,Karma Resorts,Other,Past Guests,Karma Bavaria,PF304831,Mrs,Alexandra,Babic,Female,...,2025-10-12 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
24,KB045761,Karma Resorts,Other,Past Guests,Karma Bavaria,PF279394,Herr,Maximilian,Goebl,Male,...,2025-10-12 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
27,KB053726,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332643,Frau,Tazmeen,Karim,Unknown,...,2025-10-12 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
28,KB053676,Karma Resorts,Other,Past Guests,Karma Bavaria,PF118055,Herr,Tahir,Karim,Male,...,2025-10-12 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
46,KK075813,Karma Resorts,Other,Past Guests,Karma Kandara,PF114997,Mr,Tan Kok,Seng,Male,...,2025-10-13 00:00:00,4,HOUSE_USE,V_OWNER,COMP,4,0,0,Email and Phone,NO
65,KSM017259,Karma Resorts,Other,Past Guests,Karma St Martins,PF318787,Mr,Timothy,Jones,Unknown,...,2025-10-13 00:00:00,1,COMP,STAFF,COMP,2,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK080879,Karma Resorts,Other,Past Guests,Karma Kandara,PF332062,Mr.,Henry Ted,Sinclair,Male,...,2025-10-12 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
1,KB051072,Karma Resorts,Other,Past Guests,Karma Bavaria,PF317005,empty,Jennifer,Brolly,Unknown,...,2025-10-12 00:00:00,2,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
2,KK075812,Karma Resorts,Other,Past Guests,Karma Kandara,PF331979,Mr.,Soon Meng,Lim,Male,...,2025-10-12 00:00:00,3,DIR,CHAT,DIGITAL,8,0,0,Email and Phone,NO
3,KK080978,Karma Resorts,Other,Past Guests,Karma Kandara,PF332295,Mr,Robert,Hunter,Male,...,2025-10-12 00:00:00,5,COMP,STAFF,COMP,1,0,0,Email Only,NO
4,KSM014640,Karma Resorts,Other,Past Guests,Karma St Martins,PF300957,empty,Sylvia,Schill,Unknown,...,2025-10-12 00:00:00,6,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,KSM015583,Karma Resorts,Other,Past Guests,Karma St Martins,PF314483,Miss,Saski,Bridges,Female,...,2025-10-19 00:00:00,1,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
62,KB052695,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325760,empty,Roberto,Bardini,Unknown,...,2025-10-19 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Phone Only,NO
63,KSM014678,Karma Resorts,Other,Past Guests,Karma St Martins,PF301497,Miss,Charlotte,Marsh,Unknown,...,2025-10-19 00:00:00,3,COMP,COMPETITION,MEDIA,2,0,0,Email and Phone,NO
64,KB053287,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328993,empty,Anastasia,Gazou,Unknown,...,2025-10-19 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_8576\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK080879,Karma Resorts,Other,Past Guests,Karma Kandara,PF332062,Mr.,Henry Ted,Sinclair,Male,...,2025-10-12,2,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
1,KB053461,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332644,Mr,Ignacio,Marini,Male,...,2025-10-12,3,FF,GSTAFF,COMP,1,0,0,Email and Phone,NO
2,KB053588,Karma Resorts,Other,Past Guests,Karma Bavaria,PF330329,,Nadja,Schittko,Unknown,...,2025-10-12,4,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
3,KB052459,Karma Resorts,Other,Past Guests,Karma Bavaria,PF324654,Herr,Stefan,Moser,Male,...,2025-10-12,1,OTA,OTA,RETAIL,6,0,0,Email and Phone,NO
4,KB053951,Karma Resorts,Other,Past Guests,Karma Bavaria,PF332769,Herr,Crishoph,Dethleffsen,Male,...,2025-10-12,2,LGR,TEL,RETAIL,2,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,KSM014678,Karma Resorts,Other,Past Guests,Karma St Martins,PF301497,Miss,Charlotte,Marsh,Unknown,...,2025-10-19,3,COMP,COMPETITION,MEDIA,2,0,0,Email and Phone,NO
62,KB053287,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328993,,Anastasia,Gazou,Unknown,...,2025-10-19,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
63,KSM015583,Karma Resorts,Other,Past Guests,Karma St Martins,PF314483,Miss,Saski,Bridges,Female,...,2025-10-19,1,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
64,KB052695,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325760,,Roberto,Bardini,Unknown,...,2025-10-19,1,OTA,OTA,RETAIL,2,1,0,Phone Only,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 81
Jumlah Data Yang Duplicate : 9
Jumlah Data Setelah Hapus Duplicate : 72
Jumlah Data Yang Termasuk Member: 6
Jumlah Data Setelah Hapus Member : 66
Jumlah Data Akhir : 66


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)